In [ ]:
## For colab runs
# from google.colab import drive
# drive.mount('/content/gdrive')

# !cp -r /content/gdrive/My\ Drive/uploads/rogozin .
# !cp rogozin/decentralized_methods-develop.zip .
# !unzip -qn decentralized_methods-develop.zip
# !mv decentralized_methods-develop/* .
# !rm decentralized_methods-develop* -rf

# !unzip -qn rogozin/vladiku.zip -d data
# !bunzip2 data/*.bz2

# !sed -ri '8d' src/utils.py

# DGM Minimal Environment 

In [ ]:
import torch
import pickle
import warnings
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_svmlight_file
from pathlib import Path

from src.objectives import ( 
    Objective,
    LeastSquares, LogRegression,
    StochLeastSquares, StochLogRegression)
from src.methods import (
    EXTRA, DIGing, DSGD,
    DAccGD, Mudag, APM1_C,
    SDAccGD, SMudag, SAPM1_C)
from src.utils import PythonGraph, expected_lambda2

# from src.sparse import objectives as obj
warnings.simplefilter('ignore')

In [ ]:
TASK = LeastSquares
# TASK = LogRegression 

DDIR = 'logreg_solutions' if TASK == LogRegression else 'least_squares_solutions' 
DSDIR = Path('data/YearPredictionMSD')
DDIR = Path(DDIR)

num_nodes = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
soldir =  DDIR / DSDIR.name

A, b = load_svmlight_file(str(DSDIR))
A = torch.Tensor(A.todense()).to(device)
b = torch.Tensor(b).to(device)

In [ ]:
fname = list(soldir.iterdir())[0].name
sigma = float(fname.split('=')[1])

with open(soldir/fname, 'rb') as file:
    f_star = pickle.load(file)['func_star']

In [ ]:
# For simulating a graph evolution,
# only graphs like 'erdos_renyi' is appropriate

p = .68
graph = 'random_geometric'
# graph = 'erdos_renyi'
# graph = 'path'
# graph = 'cycle'
# graph = 'complete'

F = TASK(A, b, num_nodes, sigma)
X0 = torch.zeros(num_nodes, A.size(1)).to(device)

In [ ]:
L = torch.svd(A)[1][0] ** 2 / (4*len(A))
kappa_g = torch.svd(F.A)[1][:, 0].mean() / sigma

In [ ]:
gen = lambda : PythonGraph(F, graph, p).gen()[1]
E_s2,_ = expected_lambda2(gen, 5000)

####
# Fixing seed doesn't really make a difference
####
# torch.manual_seed(123)  #  I don't remember whether I use torch random numbers anywhere
# random.seed(123)  #  networkx depends on lib random
# graphs = [PythonGraph(F, graph, p).gen()[1] for _ in range(int(1e4))]

# class GraphEvolution:
#     def __init__(self, graphs):
#         self.gi = iter(graphs)
        
#     def __call__(self):
#         return next(self.gi)

In [ ]:
opts = []
opts.append(
    EXTRA(F, gen, eta=.01/F.b.norm()))

opts.append(
    DIGing(F, gen, eta=.01/F.b.norm()))

consensus_iters = 3
opts.append(
    DAccGD(F, gen, L=L, mu=sigma, con_iters=consensus_iters))

consensus_iters = 5
opts.append(
    SDAccGD(F, gen, L=L, mu=sigma, E_s2=E_s2, con_iters=consensus_iters))

consensus_iters = 2
opts.append(
    SMudag(F, gen, L=L, mu=sigma, E_s2=E_s2, con_iters=consensus_iters))

opts.append(
    SAPM1_C(F, gen, L=L, mu=sigma, E_s2=E_s2, beta=0, scale=100))  # NOTE: tune up scale

In [ ]:
# opts = []
# W = gen()
# opts.append(
#     Mudag(F, W, L=L, mu=sigma, M=L, kappa_g=kappa_g, scale=1.))
# 
# beta = 0
# opts.append(
#     APM1_C(F, W, L=L, mu=sigma, beta=beta, scale=100))  # NOTE: tune up scale
# 
# M has almost no effect on the convergence

In [ ]:
%%time
n_iters = 1000

for opt in opts:
    opt.run(X0, n_iters=n_iters);

In [ ]:
XAXIS = 'nmix'
# XAXIS = 'i'
XLIM = n_iters if XAXIS == 'i' else min([opt.logs['nmix'][-1] for opt in opts]) / 1.2

plt.figure(figsize=(8, 8))
for i, opt in enumerate(opts):
    lbl = opt.__class__.__name__
    span = np.searchsorted(opt.logs[XAXIS], XLIM)
    plt.plot(
        opt.logs[XAXIS][:span], opt.logs['fn'][:span] - f_star,
        marker=i+5, label=lbl, markevery=n_iters//(5+i*3))  # FIXME: change abs marker frequency to relative
    
plt.title('Optimization Functional Value over Iteration Number', size=20)
plt.ylabel(r'$f(\overline{x}_k) - f^*$', size=20)
plt.xlabel('communication steps', size=20)
# plt.yscale('log')
plt.grid()
plt.legend();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_residual.png')

In [ ]:
XAXIS = 'nmix'
# XAXIS = 'i'
XLIM = n_iters if XAXIS == 'i' else min([opt.logs['nmix'][-1] for opt in opts])

plt.figure(figsize=(8, 8))
for i, opt in enumerate(opts):
    lbl = opt.__class__.__name__
    span = np.searchsorted(opt.logs[XAXIS], XLIM)
    plt.plot(
        opt.logs['nmix'][:span], opt.logs['dist2con'][:span],
        marker=i+5, label=lbl, markevery=n_iters//(5+i*3))  # FIXME: change abs marker frequency to relative

plt.ylabel(r'$||(I-\frac{1}{n}11^T)X||^2$', size=20)
plt.xlabel('communication steps', size=20)
plt.yscale('log')
plt.legend()
plt.grid();

# plt.savefig('../Decentralized + inexactness/en/figures/a9a_consensus.png')